In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [3]:


# Load datasets
gdp_data = pd.read_csv(r"D:\Finsight-Bharat\GDP_and_Major_Industrial_Sectors_of_Economy_Dataset.csv")
inflation_data = pd.read_csv(r"D:\Finsight-Bharat\CPI_Inflation_India_1970_2024.csv")
trade_data = pd.read_csv(r"D:\Finsight-Bharat\Foreign_Exchange_Trade_Balance_India_1980_2024.csv")
budget_data = pd.read_csv(r"D:\Finsight-Bharat\Budget_Allocation_India_1990_2024.csv")
nifty_data = pd.read_csv(r"D:\Finsight-Bharat\NIFTY 50-20-03-2024-to-20-03-2025.csv")

# Display first few rows of each dataset
print("GDP Data:\n", gdp_data.head())
print("\nInflation Data:\n", inflation_data.head())
print("\nTrade Balance Data:\n", trade_data.head())
print("\nBudget Allocation Data:\n", budget_data.head())
print("\nGDP Growth Rate Data:\n", nifty_data.head())


GDP Data:
   Financial Year Gross Domestic Product (in Rs. Cr) at 2004-05 Prices  \
0        1951-52                                           286,147      
1        1952-53                                           294,267      
2        1953-54                                           312,177      
3        1954-55                                           325,431      
4        1955-56                                           333,766      

  Agriculture & Allied Services (in Rs. Cr.) at 2004-05 Prices  \
0                                           147,216              
1                                           151,859              
2                                           163,553              
3                                           168,361              
4                                           166,906              

  Agriculture (in Rs. Cr.) at 2004-05 Prices  \
0                                   118,877    
1                                   123,822    
2        

In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [13]:
print(gdp_data.columns)


Index(['Financial Year',
       'Gross Domestic Product (in Rs. Cr) at 2004-05 Prices',
       'Agriculture & Allied Services (in Rs. Cr.) at 2004-05 Prices',
       'Agriculture (in Rs. Cr.) at 2004-05 Prices',
       'Industry (in Rs. Cr.) at 2004-05 Prices',
       'Mining and Quarrying (in Rs. Cr.) at 2004-05 Prices',
       'Manufacturing (in Rs. Cr.) at 2004-05 Prices',
       'Services (in Rs. Cr.) at 2004-05 Prices',
       'Agri-culture & Allied Services -Share to Total GDP',
       'Agriculture - Share to Total GDP', 'Industry - Share to Total GDP',
       'Mining and Quarrying - Share to Total GDP',
       'Manufacturing - Share to Total GDP', 'Services - Share to Total GDP',
       'Gross Domestic Product - % Growth Rate (YoY)',
       'Agri-culture & Allied Services - % Growth Rate (YoY)',
       'Agriculture - % Growth Rate (YoY)', 'Industry - % Growth Rate (YoY)',
       'Mining and Quarrying - % Growth Rate (YoY)',
       'Manufacturing - % Growth Rate (YoY)',
       'S

In [31]:
def load_and_preprocess_data(file_path):
    df = pd.read_csv(file_path)
    df["Financial Year"] = df["Financial Year"].str.split("-").str[0].astype(int)
    df.iloc[:, 1:] = df.iloc[:, 1:].replace(",", "", regex=True).astype(float)
    df.fillna(method="ffill", inplace=True)
    return df

In [ ]:
def train_models(df):
    X = df[["Financial Year"]]
    y = df[["Agriculture & Allied Services (in Rs. Cr.) at 2004-05 Prices",
            "Industry (in Rs. Cr.) at 2004-05 Prices",
            "Mining and Quarrying (in Rs. Cr.) at 2004-05 Prices",
            "Manufacturing (in Rs. Cr.) at 2004-05 Prices",
            "Services (in Rs. Cr.) at 2004-05 Prices"]]
    
    models = {}
    for sector in y.columns:
        model = LinearRegression()
        model.fit(X, y[sector])
        models[sector] = model
    
    return models

In [32]:
def predict_gdp(models, start_year, end_year):
    future_years = np.array(range(start_year, end_year + 1)).reshape(-1, 1)
    predictions = {sector: models[sector].predict(future_years) for sector in models.keys()}
    percentage_increase = {}
    for sector, values in predictions.items():
        initial = values[0]
        final = values[-1]
        increase = ((final - initial) / initial) * 100 if initial != 0 else 0
        percentage_increase[sector] = increase
    
    report_data = {"Year": future_years.flatten()}
    for sector, values in predictions.items():
        report_data[sector] = values.flatten()
    report_df = pd.DataFrame(report_data)
    report_df["Percentage Increase"] = report_df.iloc[:, 1:].pct_change().sum(axis=1) * 100
    
    file_name = f"GDP_Prediction_Report_{start_year}_to_{end_year}.csv"
    report_df.to_csv(file_name, index=False)
    print(f"Report saved as {file_name}")
    
    print("\nPredicted GDP Contribution by Sector:")
    print(report_df)
    print("\nPercentage Increase from Start to End Year:")
    for sector, increase in percentage_increase.items():
        print(f"  {sector}: {increase:.2f}%")
    
    return future_years.flatten(), predictions, percentage_increase, file_name


In [33]:
def visualize_predictions(df, future_years, predictions):
    plt.figure(figsize=(12, 6))
    for sector, values in predictions.items():
        plt.plot(df["Financial Year"], df[sector], label=f"Historical {sector}", linestyle="dotted")
        plt.plot(future_years, values, label=f"Predicted {sector}", linestyle="solid")
    plt.xlabel("Year")
    plt.ylabel("GDP Contribution (in Rs. Cr.)")
    plt.title("Sector-wise GDP Contribution Prediction")
    plt.legend()
    plt.grid(True)
    plt.show()

In [28]:
# Example Usage
file_path = "GDP_and_Major_Industrial_Sectors_of_Economy_Dataset.csv"
df = load_and_preprocess_data(file_path)
models = train_models(df)

C:\Users\PRAKASH\AppData\Local\Temp\ipykernel_2992\12403561.py:5: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method="ffill", inplace=True)
C:\Users\PRAKASH\AppData\Local\Temp\ipykernel_2992\12403561.py:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.fillna(method="ffill", inplace=True)


In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pdfkit
import os
import pdfkit

# Load GDP Data
def load_gdp_data(file_path):
    df = pd.read_csv(file_path)
    df.dropna(inplace=True)
    return df

# Train Polynomial Regression Model
def train_gdp_model(df):
    models = {}
    for sector in df.columns[1:]:
        X = df[['Year']].values
        y = df[sector].values
        models[sector] = np.poly1d(np.polyfit(X.flatten(), y, 2))  # Polynomial Regression
    return models

# Predict GDP
def predict_gdp(models, start_year, end_year):
    future_years = np.arange(start_year, end_year + 1)
    predictions = {sector: models[sector](future_years) for sector in models}
    return future_years, predictions

# Visualization Fix (No Overlap & More Readable)
def visualize_predictions(df, future_years, predictions, save_path="gdp_plot.png"):
    plt.figure(figsize=(12, 6))  # Increased figure size
    colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k']  # Use distinct colors
    for i, (sector, values) in enumerate(predictions.items()):
        plt.plot(future_years, values, linestyle='-', marker='o', markersize=4, color=colors[i % len(colors)], label=f"{sector}")
    
    plt.xlabel("Year", fontsize=12)
    plt.ylabel("GDP Contribution", fontsize=12)
    plt.title("GDP Sector-wise Forecast", fontsize=14, fontweight="bold")
    plt.legend(loc="upper left", bbox_to_anchor=(1, 1))  # Move legend outside
    plt.grid(alpha=0.3)
    plt.xticks(rotation=45)  # Rotate x-axis labels for better visibility
    plt.tight_layout()  # Prevent overlapping
    plt.savefig(save_path)
    plt.close()
    return save_path

# Generate PDF Report with Download Button
# Specify the correct path to wkhtmltopdf
pdfkit_config = pdfkit.configuration(wkhtmltopdf=r"C:\Program Files\wkhtmltopdf\bin\wkhtmltopdf.exe")

def generate_pdf_report(selected_data, visualization_path, report_path="GDP_Report.pdf"):
    html_content = f"""
    <html>
    <head>
        <title>GDP Report</title>
        <style>
            body {{ font-family: Arial, sans-serif; line-height: 1.6; }}
            h1, h2 {{ color: #2C3E50; }}
            table {{ width: 100%; border-collapse: collapse; margin-top: 20px; }}
            th, td {{ border: 1px solid #ddd; padding: 8px; text-align: left; }}
            th {{ background-color: #3498DB; color: white; }}
            img {{ max-width: 100%; height: auto; margin-top: 20px; }}
        </style>
    </head>
    <body>
        <h1>GDP Sector-wise Analysis Report</h1>
        <h2>Selected Data:</h2>
        {selected_data.to_html(index=False)}
        <h2>Visualization:</h2>
        <img src='{visualization_path}' width='700px'>
    </body>
    </html>
    """

    with open("report.html", "w", encoding="utf-8") as file:
        file.write(html_content)

    # Generate PDF with configuration
    pdfkit.from_file("report.html", report_path, configuration=pdfkit_config)
    print(f"✅ PDF report generated: {report_path}")
    return report_path

# Process User Request with Error Handling
def process_gdp_request(df, models, start_year, end_year):
    min_year, max_year = df["Year"].min(), df["Year"].max()

    # Ensure selected years are valid
    if start_year < min_year or end_year > max_year:
        print(f"🔴 Error: Selected years should be between {min_year} and {max_year}.")
        return {}

    # Extract historical data
    selected_data = df[(df["Year"] >= start_year) & (df["Year"] <= end_year)]

    # Generate predictions
    future_years, predictions = predict_gdp(models, start_year, end_year)

    # Generate visualization
    visualization_path = visualize_predictions(df, future_years, predictions)

    # Generate PDF Report
    report_path = generate_pdf_report(selected_data, visualization_path)

    return {
        "data": selected_data,
        "predictions": predictions,
        "visualization": visualization_path,
        "report": report_path
    }

# Example Usage
file_path = r"D:\Finsight-Bharat\GDP_Prediction_Report_2012_to_2020.csv"  # Update with actual dataset
df = load_gdp_data(file_path)
models = train_gdp_model(df)

# User-defined range (Modify these values as needed)
start_year = 2012
end_year = 2020

result = process_gdp_request(df, models, start_year, end_year)

if result:  # Ensure result is not empty before accessing keys
    print("📊 Data Preview:\n", result["data"])
    print("📈 Visualization saved at:", result["visualization"])
    print("📄 Download report:", result["report"])
else:
    print("⚠️ Please select a valid year range.")


OSError: No wkhtmltopdf executable found: "C:\Program Files\wkhtmltopdf\bin\wkhtmltopdf.exe"
If this file exists please check that this process can read it or you can pass path to it manually in method call, check README. Otherwise please install wkhtmltopdf - https://github.com/JazzCore/python-pdfkit/wiki/Installing-wkhtmltopdf